In [ ]:
# Music player in python

from pygame import mixer

mixer.init()
# Path of music
song = input("What song do you want to play? >")
song = song.lower()
mixer.music.load(song + ".mp3")
mixer.music.set_volume(0.5)
mixer.music.play()

print("Press 'p' to pause")
print("Press 'r' to resume")
print("Press 'v' set volume")
print("Press 'e' to exit")

while True:

    ch = input("['p','r','v','e']>>>")

    if ch == "p":
        mixer.music.pause()
        print("Music paused")
    elif ch == "r":
        mixer.music.unpause()
        print("Music resumed")
    elif ch == "v":
        v = float(input("Enter volume (0 to 10): ")) / 10
        mixer.music.set_volume(v)
        print(f'Volume set to {int(v * 10)}')
    elif ch == "e":
        mixer.music.stop()
        print("Exiting...")
        break
    else:
        print("Sorry, I don't understand that command.")

In [ ]:
import requests
from tkinter import filedialog, Tk, Menu, Listbox, Button, Frame, PhotoImage, END
import pygame
import os

# Initialize the Tkinter window
app = Tk()
app.title('Music Player')
app.geometry("500x300")

# Change the application icon
app_icon = PhotoImage(file='background.png') 
app.iconphoto(False, app_icon)

# Initialize Pygame's mixer module for playing audio
pygame.mixer.init()

# Define an event for when a song ends
SONG_END_EVENT = pygame.USEREVENT + 1
pygame.mixer.music.set_endevent(SONG_END_EVENT)

# Create a menu bar
menu_bar = Menu(app)
app.config(menu=menu_bar)

# Define global variables
playlist = []  # List to store names of songs
current_song = ""  # Store the currently playing song
is_paused = False  # Flag to indicate if music is paused

# Function to load music from a directory
def load_music():
    global current_song
    app.directory = filedialog.askdirectory()

    # Clear the current list of songs
    playlist.clear()
    song_listbox.delete(0, END)

    # Iterate through files in the directory and add MP3 files to the playlist
    for file in os.listdir(app.directory):
        name, ext = os.path.splitext(file)
        if ext == '.mp3':
            playlist.append(file)

    # Add songs to the listbox
    for song in playlist:
        song_listbox.insert("end", song)

    # Select the first song in the list by default, if there are any songs
    if playlist:
        song_listbox.selection_set(0)
        current_song = playlist[song_listbox.curselection()[0]]

# Function to play music
def play_music(event=None):
    global current_song, is_paused

    if not playlist:
        return

    # Get the selected song from the listbox
    current_selection = song_listbox.curselection()
    if current_selection:
        current_song = playlist[current_selection[0]]

    # If not paused, load and play the current song
    if not is_paused:
        pygame.mixer.music.load(os.path.join(app.directory, current_song))
        pygame.mixer.music.play()
    else:
        # If paused, unpause the music
        pygame.mixer.music.unpause()
        is_paused = False

# Function to pause music
def pause_music():
    global is_paused
    if not playlist:
        return
    pygame.mixer.music.pause()
    is_paused = True

# Function to play the next song
def next_song():
    global current_song, is_paused

    if not playlist:
        return

    try:
        # Clear previous selection and select the next song in the list
        song_listbox.selection_clear(0, END)
        next_index = (playlist.index(current_song) + 1) % len(playlist)
        song_listbox.selection_set(next_index)
        current_song = playlist[song_listbox.curselection()[0]]
        is_paused = False  # Reset paused flag for next song
        play_music()
    except:
        pass

# Function to play the previous song
def previous_song():
    global current_song, is_paused

    if not playlist:
        return

    try:
        # Clear previous selection and select the previous song in the list
        song_listbox.selection_clear(0, END)
        prev_index = (playlist.index(current_song) - 1) % len(playlist)
        song_listbox.selection_set(prev_index)
        current_song = playlist[song_listbox.curselection()[0]]
        is_paused = False  # Reset paused flag for previous song
        play_music()
    except:
        pass

# Function to check if the music has ended
def check_music_end():
    if not pygame.mixer.music.get_busy() and not is_paused and playlist:
        next_song()
    app.after(100, check_music_end)

# Create a menu for adding songs
add_songs_menu = Menu(menu_bar, tearoff=False)
add_songs_menu.add_command(label='Select Folder', command=load_music)
menu_bar.add_cascade(label='Add Songs', menu=add_songs_menu)

# Create a listbox to display songs
song_listbox = Listbox(app, bg="green", fg="white", width=100, height=13)
song_listbox.pack()

# Bind a selection event to the listbox
song_listbox.bind("<<ListboxSelect>>", play_music)

# Load images for control buttons
play_button_image = PhotoImage(file='play.png')
pause_button_image = PhotoImage(file='pause.png')
next_button_image = PhotoImage(file='next.png')
previous_button_image = PhotoImage(file='previous.png')

# Create control buttons
control_frame = Frame(app)
control_frame.pack()

play_button = Button(control_frame, image=play_button_image, borderwidth=0, command=play_music)
pause_button = Button(control_frame, image=pause_button_image, borderwidth=0, command=pause_music)
next_button = Button(control_frame, image=next_button_image, borderwidth=0, command=next_song)
previous_button = Button(control_frame, image=previous_button_image, borderwidth=0, command=previous_song)

# Arrange control buttons
previous_button.grid(row=0, column=0, padx=5)
play_button.grid(row=0, column=1, padx=5)
pause_button.grid(row=0, column=2, padx=5)
next_button.grid(row=0, column=3, padx=5)

# Start checking for the end of song event
app.after(100, check_music_end)

# Start Tkinter event loop
app.mainloop()


In [ ]:
import requests
from tkinter import filedialog, Tk, Menu, Listbox, Button, Frame, PhotoImage, END
import pygame
from pygame import mixer
import os
import random

# Initialize the Tkinter window
app = Tk()
app.title('Music Player')
app.geometry("500x300")

# Change the application icon
app_icon = PhotoImage(file='background.png') 
app.iconphoto(False, app_icon)

# Initialize Pygame's mixer module
pygame.mixer.init()
SONG_END_EVENT = pygame.USEREVENT + 1
pygame.mixer.music.set_endevent(SONG_END_EVENT)

# Define global variables
playlist = []
current_song = ""
is_paused = False
shuffle_mode = False
repeat_mode = False

# Load music from a directory
def load_music():
    global current_song
    app.directory = filedialog.askdirectory()
    playlist.clear()
    song_listbox.delete(0, END)
    for file in os.listdir(app.directory):
        name, ext = os.path.splitext(file)
        if ext == '.mp3':
            playlist.append(file)
    for song in playlist:
        song_listbox.insert("end", song)
    if playlist:
        song_listbox.selection_set(0)
        current_song = playlist[song_listbox.curselection()[0]]

# Play music
def play_music(event=None):
    global current_song, is_paused
    if not playlist:
        return
    current_selection = song_listbox.curselection()
    if current_selection:
        current_song = playlist[current_selection[0]]
    if not is_paused:
        pygame.mixer.music.load(os.path.join(app.directory, current_song))
        pygame.mixer.music.play()
    else:
        pygame.mixer.music.unpause()
        is_paused = False

# Pause music
def pause_music():
    global is_paused
    if not playlist:
        return
    pygame.mixer.music.pause()
    is_paused = True

# Play next song
def next_song():
    global current_song, is_paused
    if not playlist:
        return
    try:
        song_listbox.selection_clear(0, END)
        if shuffle_mode:
            next_index = random.randint(0, len(playlist) - 1)
        else:
            next_index = (playlist.index(current_song) + 1) % len(playlist)
        song_listbox.selection_set(next_index)
        current_song = playlist[next_index]
        is_paused = False
        play_music()
    except:
        pass

# Play previous song
def previous_song():
    global current_song, is_paused
    if not playlist:
        return
    try:
        song_listbox.selection_clear(0, END)
        prev_index = (playlist.index(current_song) - 1) % len(playlist)
        song_listbox.selection_set(prev_index)
        current_song = playlist[prev_index]
        is_paused = False
        play_music()
    except:
        pass

# Toggle shuffle mode
def toggle_shuffle():
    global shuffle_mode
    shuffle_mode = not shuffle_mode
    shuffle_button.config(highlightbackground="#FF0000" if shuffle_mode else "SystemButtonFace")

# Toggle repeat mode
def toggle_repeat():
    global repeat_mode
    repeat_mode = not repeat_mode
    repeat_button.config(highlightbackground="#FF0000" if repeat_mode else "SystemButtonFace")

# Check for end of song
def check_music_end():
    if not pygame.mixer.music.get_busy() and not is_paused and playlist:
        if repeat_mode:
            play_music()
        else:
            next_song()
    app.after(100, check_music_end)

# Create a menu for adding songs
menu_bar = Menu(app)
add_songs_menu = Menu(menu_bar, tearoff=False)
add_songs_menu.add_command(label='Select Folder', command=load_music)
menu_bar.add_cascade(label='Add Songs', menu=add_songs_menu)
app.config(menu=menu_bar)

# Create a listbox to display songs
song_listbox = Listbox(app, bg="green", fg="white", width=100, height=13)
song_listbox.pack()
song_listbox.bind("<<ListboxSelect>>", play_music)

# Load images for control buttons
play_button_image = PhotoImage(file='play.png')
pause_button_image = PhotoImage(file='pause.png')
next_button_image = PhotoImage(file='next.png')
previous_button_image = PhotoImage(file='previous.png')
shuffle_button_image = PhotoImage(file='shuffle.png')
repeat_button_image = PhotoImage(file='repeat.png')

# Create control buttons
control_frame = Frame(app)
control_frame.pack()

previous_button = Button(control_frame, image=previous_button_image, borderwidth=0, command=previous_song)
play_button = Button(control_frame, image=play_button_image, borderwidth=0, command=play_music)
pause_button = Button(control_frame, image=pause_button_image, borderwidth=0, command=pause_music)
next_button = Button(control_frame, image=next_button_image, borderwidth=0, command=next_song)
shuffle_button = Button(control_frame, image=shuffle_button_image, borderwidth=0, command=toggle_shuffle, highlightbackground="SystemButtonFace")
repeat_button = Button(control_frame, image=repeat_button_image, borderwidth=0, command=toggle_repeat, highlightbackground="SystemButtonFace")

# Arrange control buttons
previous_button.grid(row=0, column=0, padx=5)
play_button.grid(row=0, column=1, padx=5)
pause_button.grid(row=0, column=2, padx=5)
next_button.grid(row=0, column=3, padx=5)
shuffle_button.grid(row=0, column=4, padx=5)
repeat_button.grid(row=0, column=5, padx=5)

# Start checking for end of song event
app.after(100, check_music_end)

# Start Tkinter event loop
app.mainloop()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


2025-03-09 11:37:52.282 python[30182:719548] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-09 11:37:52.282 python[30182:719548] +[IMKInputSession subclass]: chose IMKInputSession_Modern
2025-03-09 11:38:23.261 python[30182:719548] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'
